### Imports And Global Variables
This Cell Contains All Needed Imports.

In [85]:
import requests
import numpy as np
import pandas as pd
import time
import json
from tqdm import tqdm
from access import Access, weights, Datasets
import logging
import geopandas as gpd
from geopandas.tools import sjoin
from shapely import wkt
import os

### Initiate API Key

Fill In With OSRM API key.

In [86]:
API_KEY = ""  # Replace With Actual API Key

### Initial File Setup

This Cell Contains The Code Needed To get The Input Files Cleaned, Merged, And Filtered Correctly.

- `2020_UA_COUNTY` Is from ESRIs Database.

- `tl_2020_us_county` Is from Census Bureau Database.

- `read_and_prepare_data` Reads And Cleans The Files To Ensure They Can Merge Correctly.

- `filter_by_state` Uses The State Abbreviation To State Code Dictionary To Get The File Contents For The Desired State.

- `getCounties` Gets The Files That Have Information On The Counties Demographics.

- `getLocations` Gets The File That has information On The providers.

In [87]:
import geopandas as gpd
import pandas as pd

# Global state FIPS dictionary
state_fips = pd.read_csv('/state-fips-dictionary.csv')
state_fips = {state_fips['Abbreviation']: state_fips['FIPS']}

def read_and_prepare_data():
    """
    Reads And Passes County Geographic Data

    Returns:
    pd.DataFrame: A DataFrame Of Counties
    gpd.GeoDataFrame: A GeoDataFrame Of Counties
    """
    # Load And prepare The county population data
    df = pd.read_csv('./2020_UA_COUNTY.csv', usecols=['STATE', 'COUNTY', 'POP_COU'])
    df['STATE'] = df['STATE'].astype(str).str.zfill(2)  # Ensure state codes are two digits
    df['COUNTY'] = df['COUNTY'].astype(str).str.zfill(3)  # Ensure county codes are three digits
    df['GEOID'] = df['STATE'] + df['COUNTY']
    # Removing ',' from The population To create an integer
    df['Pop'] = df['POP_COU'].str.replace(',', '').astype(int) 
    
    # Load And prepare The shapefile data
    shp = gpd.read_file('./tl_2020_us_county/tl_2020_us_county.shp')
    shp['GEOID'] = shp['GEOID'].astype(str)

    return df, shp


def filter_by_state(shp, df, state):
    """
    Merges And Filters Geographic Files
    
    Args:
    shp (gpd.GeoDataFrame): A Geopandas Dataframe Of County Data
    df (pd.DataFrame): A Dataframe Of County Data
    state (str): A State Abbreviation

    Returns:
    pd.DataFrame: A DataFrame Of Counties
    """
    # Merge And filter The data based on state Abbreviation
    merged_data = shp.merge(df, on='GEOID', how='inner')
    if state in state_fips:
        merged_data = merged_data[merged_data['STATE'] == state_fips[state]]
    elif state == 'AR_LA_MS':
        states = ['05', '22', '28']
        merged_data = merged_data[merged_data['STATE'].isin(states)]
    
    return merged_data

def getCounties(state):
    """
    Reads And Saves Speicifed Geographic Data
    
    Args:
    state (str): A State Abbreviation

    Returns:
    pd.DataFrame: A DataFrame Of Counties
    """
    df, shp = read_and_prepare_data()
    filtered_data = filter_by_state(shp, df, state)

    # Creating a GeoDataFrame
    geo_df = gpd.GeoDataFrame(
        filtered_data, 
        geometry=gpd.points_from_xy(filtered_data['INTPTLON'].astype(float), filtered_data['INTPTLAT'].astype(float))
    )
    geo_df = geo_df[['GEOID', 'Pop', 'geometry']]
    
    return geo_df

def getLocations(state, type):
    """
    Reads And Saves Speicifed Provider Data For A Specified State
    
    Args:
    state (str): A State Abbreviation
    type (str): A Type Of Provider

    Returns:
    pd.DataFrame: A DataFrame Of Providers
    """
    # Getting The Providers Data
    df = gpd.read_file(f'./{type}/Input/{state}_{type}.geojson')
    return df

FileNotFoundError: [Errno 2] No such file or directory: '/state-fips-dictionary.csv'

### Mapping Routes Using OSRM

This Cell Uses OSRM To Map Sources To destination.

- `extract_coords` Function Gets X, Y Coordinates From Point Object.

- `call_osrm_api` Function Takes Control Of The Flow To And From OSRM's API. 

- `get_distances` Populates The Distance Matrix.

- `mapRoutes` Gets The Input Files And Starts The Process Of Mapping The Routes. Once The Distance Matrix Has Been Returned, It Melts It For Ease Of Use.

- Hosting An Instance Of OSRM Is Also A Valueable Option. 

- *Instructions For That Can Be Found Here: https://github.com/Project-OSRM/osrm-backend*

In [88]:
def extract_coords(df):
    """
    Extracts X,Y From Point Object.
    
    Args:
    df (GeoDataFrame): A GeoDataFrame With A 'geometry' Column.

    Returns:
    numpy.ndarray: An Array Of X And Y Coordinates.
    """
    # Getting X,Y from point object
    df['X'] = df['geometry'].apply(lambda p: p.x)
    df['Y'] = df['geometry'].apply(lambda p: p.y)
    return df[['X', 'Y']].to_numpy()

def call_osrm_api(src_coords, dest_coords, sources, destinations, delay, max_retries=3):
    """
    Call The OSRM API To Get Distances Between Source And Destination Coordinates.

    Args:
    src_coords (numpy.ndarray): Array Of Source Coordinates.
    dest_coords (numpy.ndarray): Array Of Destination Coordinates.
    sources (str): String Of Source Indices For OSRM API.
    destinations (str): String Of Destination Indices For OSRM API.
    delay (int): Delay Between API Calls in Seconds.
    max_retries (int): Maximum Number Of Retries For The API Call.

    Returns:
    list: A List Of Distance Values.
    """
    coords = np.vstack((src_coords, dest_coords))
    coordinates_str = ";".join([f"{x},{y}" for x, y in coords])
    osrm_endpoint = f"http://router.project-osrm.org/table/v1/driving/{coordinates_str}?sources={sources}&destinations={destinations}&annotations=distance"
    headers = requests.utils.default_headers()
    headers.update({'User-Agent': 'My User Agent 1.0'})

    attempts = 0
    while attempts < max_retries:
        try:
            response = requests.get(osrm_endpoint, headers=headers)
            response.raise_for_status()
            return json.loads(response.content)['distances']
        except requests.exceptions.RequestException as err:
            print(f"Attempt {attempts + 1} failed: {err}")
            attempts += 1
            time.sleep(delay)

    raise Exception(f"Failed to get distances after {max_retries} attempts for sources: {sources} and destinations: {destinations}.")



def get_distances(src_coords, dest_coords, src_batch_size=10, dest_batch_size=50, delay=1):
    """
    Get Distances Between Source And Destination Coordinates Using OSRM API.

    Args:
    src_coords (numpy.ndarray): Array Of Source Coordinates.
    dest_coords (numpy.ndarray): Array Of Destination Coordinates.
    src_batch_size (int): Batch Size For Source Coordinates.
    dest_batch_size (int): Batch Size For Destination Coordinates.
    delay (int): Time Delay Between API Calls In Seconds.

    Returns:
    numpy.ndarray: A Matrix Of Distance Values.
    """
    # Initialize Matrix
    distanceMat = np.zeros((len(src_coords), len(dest_coords)))
    # Iterate Through Coordinates
    for i in tqdm(range(0, len(src_coords), src_batch_size)):
        batch_src_coords = src_coords[i:i + src_batch_size]
        sources = ";".join(map(str, range(len(batch_src_coords))))
        for j in range(0, len(dest_coords), dest_batch_size):
            batch_dest_coords = dest_coords[j:j + dest_batch_size]
            destinations = ";".join(map(str, range(len(batch_src_coords), len(batch_src_coords) + len(batch_dest_coords))))
            distances = call_osrm_api(batch_src_coords, batch_dest_coords, sources, destinations, delay)
            if distances is not None:
                distanceMat[i:i + src_batch_size, j:j + dest_batch_size] = distances
            time.sleep(delay)
    return distanceMat


def mapRoutes(state, ID, type):
    """
    Maps Routes Between Counties And Locations Of A Specified Provider Within A State.

    Args:
    state (str): State Abbreviation.
    ID (str): ID Column Of Provider.
    type (str): Type Of Locations To Be Mapped.

    Returns:
    None: Saves The Result To A CSV File.
    """

    df1 = getCounties(state)
    df2 = getLocations(state, type)

    coords1 = extract_coords(df1)
    coords2 = extract_coords(df2)
    
    print(f'{state} has {len(coords1)} counties And {len(coords2)} {ID}s.')

    # Parameters To Speed Up OSRM (change these as needed)
    src_batch_size = 100
    dest_batch_size = 100
    delay = 2

    distanceMat = get_distances(coords1, coords2, src_batch_size, dest_batch_size, delay)

    distance_df = pd.DataFrame(distanceMat, index=df1['GEOID'], columns=df2[f'{ID}'])
    # Melting The dataframe For Ease Of Use
    melted_df = pd.melt(distance_df.reset_index(), id_vars='GEOID', value_vars=distance_df.columns, var_name=f'destination ({type})', value_name='cost (distance)')
    melted_df = melted_df.rename(columns={'GEOID': 'source (GEOID)'})
    melted_df.to_csv(f'./{type}/Output/{state}_melted_{type}_Distance.csv', index=False)


### Getting The GEOID From The Provider's Coordinates

This Cell Focuses On Using The Census Bureau API To Get The GEOID Or CountyFIPS Of Each County That there Is A Provider. This Is Done Using The Coordaintes Of Each Provider. 

- `get_geoid_from_coords` Takes In A Set Of Coordinates, Connects To The API, And Returns The GEOID At The County Level For Those Coodinates.

In [89]:
def get_geoid_from_coords(latitude, longitude):
    """
    Retrieves The GEOID For Given Latitude And Longitude Using The Census Bureau's Geocoding API.
    
    Args:
    latitude (float): Latitude Of The Provider Location.
    longitude (float): Longitude Of The Provider Location.

    Returns:
    str: GEOID Of The location.
    """
    base_url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        "x": longitude,
        "y": latitude,
        "benchmark": "Public_AR_Census2020",
        "vintage": "Census2010_Census2020",
        "format": "json",
        "key": API_KEY
    }
    # Making Request To API
    response = requests.get(base_url, params=params)
    data = response.json()
    try:
        # Parsing And returning results
        geoid = data['result']['geographies']['Counties'][0]['GEOID']
        return geoid
    except (KeyError, IndexError):
        print("Could not find GEOID for The provided coordinates.")
        return None

### Supply And Demand Matrices

This Cell Generates 3 Tables:

- `getProviders` Creates A Supply Table Based On The Number Of Providers Per County (when observing Hospitals It Is Number Of Hospital Beds Rather Than Number Of providers)

- `getTimes` Uses Both OSRM And Cenus Bureau API Outputs To Create A File That Will Show `[Source, Destination, Time]` *Source And Destination Are GEOIDs*

- `getPop` Simply Gets The Population Of Each County. 

These Tables Are Used For The Input Of RAAM.

In [90]:
def getProviders(provider, col, type, state):
    """
    Aggregates Provider Data By GEOID And Merges With County Data.
    
    Args:
    provider (DataFrame): Provider Data.
    col (str): ID Column In Provider Data.
    type (str): Type Of Provider.
    state (str): State Abbreviation.

    Returns:
    None: Saves The Result To A CSV File.
    """
    t1 = getCounties(state)
    t2 = provider

    t1['GEOID'] = t1['GEOID'].astype(int)
    t2['GEOID'] = t2['GEOID'].astype(int)

    # To Make Sure Hospitals Are Counted By Beds And Not Providers
    if type == 'Hospitals':
        doctor_counts = t2.groupby('GEOID')['Licensed_A'].sum().reset_index()
        doctor_counts = doctor_counts.rename(columns={'Licensed_A': f'{type}'})
    else:
        doctor_counts = t2.groupby('GEOID')[f'{col}'].count().reset_index()
        doctor_counts = doctor_counts.rename(columns={f'{col}': f'{type}'})

    # Joining To get population And coordinates
    t3 = pd.merge(t1, doctor_counts, on='GEOID', how='left')
    t3[f'{type}'] = t3[f'{type}'].fillna(0).astype(int)
    t3 = t3[['GEOID', 'Pop', f'{type}', 'geometry']]
    t3.to_csv(f'./{type}/SupplyDemand/{state}/{state}_{type}.csv', index=False)

    # Printing Number Of Distinct Counties With A Provider
    print(f"Total Number Of Proivers ({type}):", len(t2))
    print(f"Counties With Atleast 1 Provider ({type}):", doctor_counts['GEOID'].nunique())
    print("Total Counties", t1['GEOID'].nunique())
    unmatched_geo_ids = doctor_counts[~doctor_counts['GEOID'].isin(t1['GEOID'])]
    if len(unmatched_geo_ids) >= 1:
        print(f'Unmatched GEOIDs: {unmatched_geo_ids}')

def getTimes(provider, col, type, state):
    """
    Merges Calculated Time Data With Provider Data.

    Args:
    provider (DataFrame): Provider Data.
    col (str): ID Column In Provider Data.
    type (str): Type Of Provider.
    state (str): State Abbreviation.

    Returns:
    None: Saves The Result To A CSV File.
    """
    provider['GEOID'] = provider['GEOID'].astype(int)
    times = pd.read_csv(f'./{type}/Output/{state}_melted_{type}_Distance.csv', low_memory=False)
    times['source (GEOID)'] = times['source (GEOID)'].astype(int)

    # For These Providers, Coordinates Are Used As Keys
    if type in ['MentalHealth', 'PWMH']:
        # To Ensure The Merge Is Consistent The Coordinates Go Back To A Point Object
        times['destination_geom'] = times['destination (geometry)'].apply(wkt.loads)
        times = gpd.GeoDataFrame(times, geometry='destination_geom')
        t3 = sjoin(times, provider[['GEOID', 'geometry']], op='intersects', how='left')
        t3 = t3[['source (GEOID)', 'GEOID', 'cost (distance)']]
    else:
        t3 = pd.merge(times, provider, left_on=f'destination ({type})', right_on=col, how='left')

    t3 = t3[['source (GEOID)', 'GEOID', 'cost (distance)']]
    t3.rename(columns={'GEOID': 'destination (GEOID)'}, inplace=True)
    t3 = t3.drop_duplicates(subset=['source (GEOID)', 'destination (GEOID)'])
    t3.to_csv(f'./{type}/SupplyDemand/{state}/{state}_Times_{type}.csv', index=False)

def getPop(state, type):
    """
    Retrieves Population Data For A State And Type Of Service.
    
    Args:
    state (str): State Abbreviation.
    type (str): Type Of Provider.

    Returns:
    None: Saves The Result To A CSV File.
    """
    t1 = getCounties(state)
    t1['GEOID'] = t1['GEOID'].astype(int)
    t3 = t1[['GEOID', 'Pop', 'geometry']]
    t3.to_csv(f'./{type}/SupplyDemand/{state}/{state}_Pop_{type}.csv', index=False)

### Running RAAM

This Cell Does The Analysis Of RAAM Using The Supply, Demand, And Times Tables Created. Multiple Sets Of Weights Are Used. 

- `RAAM` Runs RAAM Analysis And Saves Output

In [91]:
def RAAM(state, type):
    """
    Performs RAAM Analysis For A Specified State And Type Of Provider.

    Args:
    state (str): State Abbreviation.
    type (str): Type Of Provider.

    Returns:
    None: Saves The Result To A CSV File.
    """
    # Supressing Warning RAAM generates
    pd.options.mode.chained_assignment = None

    # Initializing Supply And Demand Tables
    pop = pd.read_csv(f'./{type}/SupplyDemand/{state}/{state}_Pop_{type}.csv')
    pop = gpd.GeoDataFrame(pop, geometry=gpd.GeoSeries.from_wkt(pop["geometry"]), crs="epsg:3005")

    provider = pd.read_csv(f'./{type}/SupplyDemand/{state}/{state}_{type}.csv')
    provider = gpd.GeoDataFrame(provider, geometry=gpd.GeoSeries.from_wkt(provider["geometry"]), crs="epsg:3005")

    times = pd.read_csv(f'./{type}/SupplyDemand/{state}/{state}_Times_{type}.csv')
    times['cost (distance)'] *= 0.000621371  # Convert meters To miles
    times = times.drop_duplicates(subset=['source (GEOID)', 'destination (GEOID)'])

    # Creating Access Object
    distanceA = Access(
        demand_df=pop, demand_index="GEOID", demand_value="Pop",
        supply_df=provider, supply_index="GEOID", supply_value=type,
        cost_df=times, cost_origin="source (GEOID)", cost_dest="destination (GEOID)", cost_name="cost (distance)"
    )

    # Changing CRS Code
    distanceA.demand_df = distanceA.demand_df.to_crs(epsg=3528)
    distanceA.supply_df = distanceA.supply_df.to_crs(epsg=3528)

    # Running RAAM With Different Sets Of Weights
    def RAAMDis(A):
        A.raam(name="raamDis", tau=60)
        A.raam(name="raam30Dis", tau=30)
        A.access_df.sort_values(by=f"raamDis_{type}").dropna().head()
        A.score(name="raamDis_combo", col_dict={f'raamDis_{type}': 0.8})
        return A

    # Saving Results
    distanceA = RAAMDis(distanceA)
    distanceA.log.setLevel(logging.WARNING)
    distanceA.norm_access_df['County'] = distanceA.norm_access_df.index.astype(str).str.slice(0, 5)
    df = distanceA.norm_access_df
    df.set_index(distanceA.norm_access_df.index)
    df.rename(columns={'raamDis_combo': f'raamDis_combo_{type}'}, inplace=True)
    df.to_csv(f"./{type}/RAAM/{state}_{type}_distance_results.csv")
    print(f'RAAM Results Saved At: ./{type}/RAAM/{state}_{type}_distance_results.csv \n\n')

### Controller 

This Cell Calls All Functions. 
- `process_state_data` Calls The Functions In The Order Needed To Run The Analysis.


In [93]:
def process_state_data(state, provider_type, id_field):
    """
    Main Function To process State Data For A Specific State And Provider Type.

    Args:
    state (str): State Abbreviation.
    provider_type (str): Type Of Provider.
    id_field (str): ID Field Name In Provider Data.

    Returns:
    None: Executes The Data Processing Workflow.
    """

    # Creating Directories
    os.makedirs(f'./{provider_type}/Output/', exist_ok=True)
    os.makedirs(f'./{provider_type}/SupplyDemand/', exist_ok=True)
    os.makedirs(f'./{provider_type}/SupplyDemand/{state}', exist_ok=True)

    os.makedirs(f'./{provider_type}/RAAM/', exist_ok=True)
    print('Mapping Routes With OSRM:')
    mapRoutes(state, id_field, provider_type)
    provider = getLocations(state, provider_type)
    # Checking To see If The Key Will Be Coordiantes
    if provider_type == 'MentalHealth' or provider_type == 'PWMH':
        # If It Is Then Use Point Objects
        provider['geometry_str'] = provider['geometry'].apply(
            lambda x: wkt.dumps(x))

    # Calling Method For Census Bureau API
    print('\nGetting GEOID From Coodinates:')

    def geoid_for_row(row):
        point = row['geometry']
        longitude = point.x
        latitude = point.y
        geoid = get_geoid_from_coords(latitude, longitude)
        return(str(int(geoid)))
    
    tqdm.pandas() 
    provider['GEOID'] = provider.progress_apply(geoid_for_row, axis=1)

    # Generating Supply And Demand Tables
    print('\nCreating Supply/Demand/Times Tables:')

    getProviders(provider, id_field, provider_type, state)
    getTimes(provider, id_field, provider_type, state)
    getPop(state, provider_type)

    print('\nRunning RAAM:')
    RAAM(state, provider_type)

### Provider Keys: ###
# Medicare: NPI
# Hospitals: HIFLD_ID
# HealthCenters: BPHC_Assig
# MentalHealth: geometry
# PWMH: geometry


process_state_data('AR', 'OBGYNsMedicaid', 'NPI')

Mapping Routes With OSRM:
AR has 75 counties And 11 NPIs.


100%|██████████| 1/1 [00:02<00:00,  2.71s/it]



Getting GEOID From Coodinates:


100%|██████████| 11/11 [00:06<00:00,  1.63it/s]



Creating Supply/Demand/Times Tables:
Total Number Of Proivers (OBGYNsMedicaid): 11
Counties With Atleast 1 Provider (OBGYNsMedicaid): 6
Total Counties 75

Running RAAM:
RAAM Results Saved At: ./OBGYNsMedicaid/RAAM/AR_OBGYNsMedicaid_distance_results.csv 


